## Assignment 3: Clustering Models

In this assignment, you will implement the K-means method and apply it to image compression. Then you will use the Gaussian Mixture Models from the scikit-learn library and select the number of clusters K using the Akaike Information Criterion (AIC) and the Bayesian Information Criterion (BIC).

#### Part 1) Implement the K-means algorithm

In [ ]:
'''
Process of K-means:
    1. Select the desired amount of clusters, K.
    2. Randomly select K distinct data points; these are the initial clusters.
    3. Measure the distance between the first data point and all the K initial clusters. For 2D, euclidian distance.
    4. Assign the first point to its nearest cluster. Do the same thing for all the remaining points.
    5. Calculate the mean of each cluster.
    6. Then we repeat the process, but this time with the mean values as the clusters (cluster points).
    The result of the clusters produced by this method is measured by the variation within the clusters.
    The model cannot "see" the clusters, it can only determine weather the clusters are good or not with 
    measuring the variation within and then sum the variation. 
    Since the initial points are chosen at random, it is good practice to repeat the above method 
    multiple times, with different randomised initial point, and for each result
    measure the variation within each cluster and determine which one produced the best clusters 
    by comparing them with each other.  

    Pick K by finding the "elbow" in an "eblow-plot". The "elbow-plot" is a plot of the reduction
    in variance per value of K.

'''


def kmeans(x, K: int, n_init: int):   
    # x: input data     
    # K: number of centroids
    # n_init: the number of initial guesses for the centroids
    # centroids: contains the centers of the clusters
    # labels: contains the cluster index for each data point 
    return centroids, labels

#### Part 2) Apply the K-means algorithm to compress images

#### Part 3) Use AIC and BIC to choose K for Gaussian Mixture Models 